In [ ]:
import os 
import pandas as pd
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt 
import seaborn as sns

# !python -m spacy download en_core_web_trf
# !python -m spacy download es_core_web_trf
# nltk.download()
import spacy 
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from chamd import ChatReader
reader = ChatReader()

# Import the Self-scripted .py for task-specific functions
import Analysis 
from Analysis import raw_chat_to_dataframe
from Analysis import summary_table
from Analysis import q1_stentence_length
from Analysis import q2_sentence_cleaning
from Analysis import q2_part_of_speech
from Analysis import q3_keyword_frequency

# Set display options for dataframes/plots
%config InlineBackend.figure_format = 'svg'
pd.options.display.max_colwidth=120

folder = "/Users/noel/Documents/GitHub/Projects_Selected/Frog Story Corpora/data_from_official_site"

## 1) Transform all transcripts into readable dataframes

a) Extract information by researcher - Spanish

b) Extract information by researcher - English

c) Combine all the extracted information by languages 

In [ ]:
# a) Extract information by researcher - Spanish
df_spa_agu = raw_chat_to_dataframe(data_folder_raw=os.path.join(folder, "Spanish-Aguilar"))
df_spa_mia = raw_chat_to_dataframe(data_folder_raw=os.path.join(folder, "Spanish-MiamiBiling"))
df_spa_orn = raw_chat_to_dataframe(data_folder_raw=os.path.join(folder, "Spanish-Ornat"))

# b) Extract information by researcher - English
df_eng_esc = raw_chat_to_dataframe(data_folder_raw=os.path.join(folder, "English-ECSC"))
df_eng_mia1 = raw_chat_to_dataframe(data_folder_raw=os.path.join(folder, "English-MiamiBiling"))
df_eng_mia2 = raw_chat_to_dataframe(data_folder_raw=os.path.join(folder, "English-MiamiMono"))
df_eng_wol = raw_chat_to_dataframe(data_folder_raw=os.path.join(folder, "English-WolfHemp"))

print("\n Data preview: one data set for SPA - Aguilar:")
display(df_spa_agu.head(3))

print("Data preview: one data set for ENG - ECSC:")
display(df_eng_esc.head(3))

# c) Combine all the extracted information by languages 
df_all_spa = pd.concat([df_spa_agu, df_spa_mia, df_spa_orn])
df_all_eng = pd.concat([df_eng_esc, df_eng_mia1, df_eng_mia2, df_eng_wol])

print("Included researchers for all records:")
print(list(df_all_spa.lan_researcher.unique()))
print(list(df_all_eng.lan_researcher.unique()))

d) Generate summary tables by languages - For combined records

In [ ]:
# d) Generate summary tables by languages - For combined records
summary_all_spa = summary_table(df_all_spa, "Spanish")
summary_all_eng = summary_table(df_all_eng, "English")

print("=====")
print("TABLE 1: Summary statistics of data sources")
print("=====")
Tab1 = pd.concat([summary_all_spa, summary_all_eng], axis=1)
Tab1

## 2) Random Sampling

a) Randomly sample 100 records from each combined record

b) Generate summary tables by languages - For sampled records

In [ ]:
# a) Randomly sample 100 records from each combined record
randomness = 2023
df_sample_spa = df_all_spa.sample(100, random_state=randomness).reset_index(drop=True)
df_sample_eng = df_all_eng.sample(100, random_state=randomness).reset_index(drop=True)

### b) Generate a summary table for the sampled records by languages 
summary_records_sample_spa = summary_table(df_sample_spa, "Spanish")
summary_records_sample_eng = summary_table(df_sample_eng, "English")

print("=====")
print("TABLE 2: Summary statistics of samples")
print("=====")
Tab2 = pd.concat([summary_records_sample_spa, summary_records_sample_eng], axis=1)
Tab2

## 3) RQ1: Average sentence length

a) Calculate the number of sentences for each child

b) Calculate the average sentence length for each child

c) Plot the distribution of average sentence length by languages

In [ ]:
# a) Calculate the number of sentences for each child
df_sample_spa["sentence_count"] = df_sample_spa.text.apply(lambda x: q1_stentence_length(x)[0])
df_sample_eng["sentence_count"] = df_sample_eng.text.apply(lambda x: q1_stentence_length(x)[0])

# b) Calculate the average sentence length for each child
df_sample_spa["sentence_length_mean"] = df_sample_spa.text.apply(lambda x: q1_stentence_length(x)[1])
df_sample_eng["sentence_length_mean"] = df_sample_eng.text.apply(lambda x: q1_stentence_length(x)[1])

# c) Plot the distribution of average sentence length by language
plot_length_distribution, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3), sharex=True, sharey=True)
plot_length_distribution.subplots_adjust(top=0.85)
plot_length_distribution.suptitle("Distribution of average sentence length (mean)", fontsize=12, x=0.5, y=0.95)

sns.histplot(ax=axes[0], data=df_sample_spa, x="sentence_length_mean", bins=10, color="blue")
sns.histplot(ax=axes[1], data=df_sample_eng, x="sentence_length_mean", bins=10, color="orange")

# Modify the plot - Change the xy labels
axes[0].set_xlabel("Spanish"); axes[1].set_xlabel("English")
axes[0].set_ylabel("Number of interviewees")

# Modify the plot - Add the line of Mean
mean_spa = round(df_sample_spa.sentence_length_mean.mean(), 1)
mean_eng = round(df_sample_eng.sentence_length_mean.mean(), 1)

axes[0].axhline(y = mean_spa, color="black", linestyle="--", linewidth=1)
axes[1].axhline(y = mean_eng, color="black", linestyle="--", linewidth=1)

axes[0].annotate(text = f"Mean= {mean_spa}", xy=(13, mean_spa+1))
axes[1].annotate(text = f"Mean= {mean_eng}", xy=(13, mean_eng-3))

d) Conduct T-test for the differences between the two languages

In [ ]:
### d) Conduct T-test for the differences 
length_ttest = stats.ttest_ind(
    df_sample_spa["sentence_length_mean"],
    df_sample_eng["sentence_length_mean"], 
)

print("=====")
print("TABLE 3: Average sentence length by language")
print("=====")

Tab3 = pd.DataFrame({
    "Language": ["Spanish", "English"],
    "Sample size": [100, 100],
    "Mean": [mean_spa, mean_eng],
    "Std": [
        round(df_sample_spa.sentence_length_mean.std(), 1), 
        round(df_sample_eng.sentence_length_mean.std(), 1)]
})

Tab3["Mean (± Std)"] = Tab3['Mean'].astype(str) +" (± " + Tab3['Std'].astype(str) + ")"
Tab3["T statistic"] = round(length_ttest.statistic, 1)
Tab3["P value"] = round(length_ttest.pvalue, 1)

Tab3.at[1, "T statistic"] = "N/A" 
Tab3.at[1, "P value"] = "N/A"

Tab3 = Tab3[["Language", "Sample size", "Mean (± Std)", "T statistic", "P value"]]
Tab3

## 4) RQ2: Most frequently used part of speech

a) Load the language model from Spacy

b) Clean the paragraphs 

c) Match the word tokens with POS and count the corresponding occurences

d) Transform the obtained occurences into readable dataframe and calculate the % 

e) Plot the distribution for the occurences by languages

In [ ]:
# a) Load the language model from Spacy
model_spa = spacy.load('es_dep_news_trf')
model_eng = spacy.load('en_core_web_trf')

# b) Clean the sentences 
clean_spa = q2_sentence_cleaning(df_dample = df_sample_spa, spa_or_eng_model = model_spa)
clean_eng = q2_sentence_cleaning(df_dample = df_sample_eng, spa_or_eng_model = model_eng)

# c) Match the word tokens with POS and count the corresponding occurences
pos_spa = q2_part_of_speech(cleaned_sentences_list = clean_spa, spa_or_eng_model = model_spa)
pos_eng = q2_part_of_speech(cleaned_sentences_list = clean_eng, spa_or_eng_model = model_eng)

pos_match_spa, pos_count_spa = pos_spa[0], pos_spa[1]
pos_match_eng, pos_count_eng = pos_eng[0], pos_eng[1]

In [ ]:
# d) Transform the obtained occurences into readable dataframe and calculate the % 
df_pos_spa = pd.DataFrame(list(pos_count_spa.items())).rename(columns={0:"POS_spa", 1:"Count"})
df_pos_spa = df_pos_spa.sort_values(by = "Count", ascending=False).reset_index(drop = True)
df_pos_spa["Count%"] = round(df_pos_spa["Count"] / df_pos_spa["Count"].sum() * 100, 2)
df_pos_spa["pos_count_order"] = list(df_pos_spa.index + 1)

df_pos_eng = pd.DataFrame(list(pos_count_eng.items())).rename(columns={0:"POS_eng", 1:"Count"})
df_pos_eng = df_pos_eng.sort_values(by = "Count", ascending=False).reset_index(drop = True)
df_pos_eng["Count%"] = round(df_pos_eng["Count"] / df_pos_eng["Count"].sum() * 100, 2)
df_pos_eng["pos_count_order"] = list(df_pos_eng.index + 1)

# d) Plot the distribution for the occurences by languages
plot_pos_distribution, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 6), sharey=True)
plot_pos_distribution.suptitle("Distribution of part of speech (occurences)", fontsize=12, x=0.5, y=0.92)

sns.barplot(ax=axes[0], data=df_pos_spa, x="POS_spa", y="Count%", color='blue')
sns.barplot(ax=axes[1], data=df_pos_eng, x="POS_eng", y="Count%", color='orange')

# Modify the plot - Change the xy labels
axes[0].set_xlabel(""), axes[0].set_ylabel("Spanish (%)")
axes[1].set_xlabel(""), axes[1].set_ylabel("English (%)")

axes[0].set_xticklabels(labels=list(df_pos_spa.POS_spa), rotation=20)
axes[1].set_xticklabels(labels=list(df_pos_eng.POS_eng), rotation=20)

## 5) RQ3: Usage frequenies of interested keywords

a) Load the language model from Spacy

b) Clean the paragraphs 

c) Count the occurences of each interested keyword

In [ ]:
# a) Load the language model from Spacy
model_spa = spacy.load('es_dep_news_trf')
model_eng = spacy.load('en_core_web_trf')

# b) Clean the paragraphs 
clean_spa = q2_sentence_cleaning(df_dample = df_sample_spa, spa_or_eng_model = model_spa)
clean_eng = q2_sentence_cleaning(df_dample = df_sample_eng, spa_or_eng_model = model_eng)

# c) Count the occurences of each interested keyword
keywords_eng = ["hi", "Noel", "hello"]
test = q3_keyword_frequency(clean_eng, keywords_eng)
test